In [1]:
import pandas as pd
import json
import ndjson

In [2]:
input_files = [
    '/home/jovyan/work/dufourspitze_16/data/ex_data_from_table_1.ndjson',
    '/home/jovyan/work/dufourspitze_16/data/ex_data_from_table_2.ndjson',
    '/home/jovyan/work/dufourspitze_16/data/ex_data_from_table_3.ndjson'
]

In [3]:
def load_data(file_name):

    ### open the file
    with open(file_name,'r') as f:
        data = ndjson.load(f)

    ### convert the list of strings to json objects
    data = [json.loads(i) for i in data]

    ### turn it into a dataframe
    df = pd.DataFrame(data)

    duplicates = {}
    for col in df.columns:
        duplicates[col] = df[col].duplicated().any()
        if len(duplicates[col]) > 0:
            print('duplicates found in column:', col)

    ### remove duplicates, if any
    df = df.drop_duplicates(subset='reaction number', keep='first')

    return df.set_index('reaction number')

In [5]:
dfs_list = []

for ifile in input_files:

    print('processing file', ifile)

    df = load_data(ifile)

    dfs_list.append(df)

processing file /home/jovyan/work/dufourspitze_16/data/ex_data_from_table_1.ndjson
processing file /home/jovyan/work/dufourspitze_16/data/ex_data_from_table_2.ndjson
processing file /home/jovyan/work/dufourspitze_16/data/ex_data_from_table_3.ndjson


In [10]:
df = pd.concat(dfs_list, axis=1)
df.head()

,reaction,reaction rate,description
reaction number,,,
429,Tumor.TC1 -> Tumor.T_Recover_Can_Dead,NaN,NaN
430,Tumor.TC2 + Tumor.Effector_TT -> Tumor.TC2,NaN,NaN
431,Tumor.Teff_EngMDSC -> Tumor.Teff_EngMDSC + Tum...,NaN,NaN
432,Tumor.Teff_EngMDSC -> Tumor.Teff_EngMDSC + Tum...,NaN,NaN
433,Tumor.Teff_EngMDSC -> Tumor.Teff_EngMDSC + Tum...,NaN,NaN


In [11]:
df = load_data('/home/jovyan/work/dufourspitze_16/data/data_from_table_1.ndjson')
df.head()

,reaction
reaction number,
1,[Blood-Lymph].CTLA4_mabB <-> Lymph_Node.CTLA4_mab
2,[Blood-Lymph].CTLA4_mabB <-> Peripheral.CTLA4_...
3,[Blood-Lymph].CTLA4_mabB <-> Peripheral.CTLA4_...
4,[Blood-Lymph].CTLA4_mabB <-> Tumor.CTLA4_mabt
5,[Blood-Lymph].CTLA4_mabB -> null


In [13]:
df = df.reset_index(drop=False)

In [14]:
len(df), len(df['reaction number'].unique())

(460, 448)

In [16]:
df = df.drop_duplicates(subset='reaction number', keep='first')